# Convolutional Neural Networks

## Project: Dog Breed Classifier

---

### Domain background

Image recognition and classification has always been an important investigation field for computer vision. It has gained a lot of popularity and advances since the introduction of Deep Learning.
Most investigations are focusing on using Convolutional Neural Networks (CNN)s to the fact that they are capable of recognizing and classifying a wide variety of objects and images. A lot of research has gone in to the training of CNNs on a variety of datasets, most focusing on the optimization of a particular network’s performance. 

### Problem Statement

There are many different dog breeds around the world, and some of them with similar physical characteristics. Being able to recognize each dog breed is a challenging task even for humans, therefore the necessity of using machine learning for performing this task. A convolutional neural network (CNN) will be used to accomplish this challenge.
The aim of this project is to build a pipeline to process real-world, user-supplied images. Given an image of a dog, the algorithm will identify an estimate of the canine’s breed. If supplied an image of a human, the code will identify the resembling dog breed. If neither a dog nor a human is detected, then an error message is output. 

Some sample output of the algorithm is provided below:

![Sample Dog Output](images/sample_dog_output.png)

![Sample Human Output](images/sample_human_output.png)


### Project Workflow

The project workflow to approach the solution is the following: 

* [Step 0](#step0): Define Imports
* [Step 1](#step1): Import Datasets
* [Step 2](#step2): Data Exploration - Human dataset
* [Step 3](#step3): Data Exploration - Dog breeds dataset
* [Step 4](#step4): Create Utilities Functions
* [Step 5](#step5): Create Benchmark Model
* [Step 6](#step6): Create Proposed Model
* [Step 7](#step7): Model Refinement
* [Step 8](#step8): Model Evaluation and Validation

<a id='step0'></a>
## Step 0: Define Imports

All necessary imports for the projects are defined.

In [ ]:
# Define all the imports
import cv2  
import numpy as np
import sys
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F

from glob import glob
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from PIL import Image
from PIL import ImageFile
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

ImageFile.LOAD_TRUNCATED_IMAGES = True